In [1]:
import numpy as np
from numpy import linalg
from sympy import *

In [2]:
# x1 = symbols('x1')
# x2 = symbols('x2')

# f1 = simplify('x2^2')
# f2 = simplify('1 - x1^2')

# g1 = Abs(simplify('1 - x1'))
# g2 = Abs(simplify('x2'))

# x0 = np.array([0, 0])

x1 = symbols('x1')
x2 = symbols('x2')

f1 = simplify('3*(x1^2)*x2 - x2^2 - 1')
f2 = simplify('x1^4 + x1*(x2^3) - 1')

g1 = Abs(simplify('x1 - 1'))
g2 = Abs(simplify('x2'))

x0 = np.array([1, 1])


# x1 = symbols('x1')
# x2 = symbols('x2')
# x3 = symbols('x3')

# f1 = simplify('(x3^2)*(1 - x2) - x1*x2')
# f2 = simplify('(x3^2)*(x1^3 - x1) - x2^2')
# f3 = simplify('6*x1*(x2^3) + (x2^2)*(x3^2) - x1*(x2^2)*x3')

# g1 = Abs(simplify('x2 - x3^2'))
# g2 = Abs(simplify('3*(x2^2) - x3^2 + 1'))
# g3 = Abs(simplify('x1 + x3 - x2'))

# x0 = np.array([0, 0, 0])

In [3]:
x = np.array([x1, x2])
F = np.array([f1, f2])
G = np.array([g1, g2])
H = np.array([f1 + g1, f2 + g2])

In [4]:
print(F)
print(G)
print(H)

[3*x1**2*x2 - x2**2 - 1 x1**4 + x1*x2**3 - 1]
[Abs(x1 - 1) Abs(x2)]
[3*x1**2*x2 - x2**2 + Abs(x1 - 1) - 1 x1**4 + x1*x2**3 + Abs(x2) - 1]


In [5]:
# x = np.array([x1, x2, x3])
# F = np.array([f1, f2, f3])
# G = np.array([g1, g2, g3])
# H = np.array([f1 + g1, f2 + g2, f3 + g3])

In [6]:
eps = 10 ** -7

In [7]:
F_0 = lambda x0: np.array([F[i].subs(dict(zip(x, x0))) for i in range(len(F))], dtype = 'float')
G_0 = lambda x0: np.array([G[i].subs(dict(zip(x, x0))) for i in range(len(G))], dtype = 'float')
H_0 = lambda x0: np.array([H[i].subs(dict(zip(x, x0))) for i in range(len(H))], dtype = 'float')

In [8]:
def F_der_0(x0):
    F_der = np.array([[diff(F[i], x[j]) for j in range(len(x))] for i in range(len(F))])
    return np.array([[F_der[i][j].subs(dict(zip(x, x0))) for j in range(len(F_der[i]))] for i in range(len(F_der))], dtype = 'float')

In [9]:
def DividedDifferenceOfOperator(F, x1, x2):
    assert(len(x1) == len(x2))
    arr = []
    for i in range(len(F)):
        subarr = []
        for j in range(len(x1)):
            params1 = [*x1[0:j+1], *x2[j+1:len(x2)+1]]
            params2 = [*x1[0:j], *x2[j:len(x2)+1]]
            result = (F[i].subs(dict(zip(x, params1))) - F[i].subs(dict(zip(x, params2)))) / (x1[j] - x2[j])
            subarr.append(result)
        
        arr.append(subarr)
    
    return np.array(arr, dtype = 'float')

In [10]:
def NewtonPotraOperatorDecompositionMethod(x0, x, F, G, H):
    # element at -1
    x_1 = x0 - 10 ** -4

    #element at -2
    x_2 = x0 - 2 * (10 ** -4)

    I = 0
    matrix = F_der_0(x0) + DividedDifferenceOfOperator(G, x0, x_1) + DividedDifferenceOfOperator(G, x_2, x0) - DividedDifferenceOfOperator(G, x_2, x_1)
    
    print(np.dot(np.linalg.inv(matrix), H_0(x0)))
    x1 = x0 - np.dot(np.linalg.inv(matrix), H_0(x0))
    print("{0} Ітерація: Значення H = {1} у точці x = {2}".format(I + 1, H_0(x1), x1))
    I += 1

    while True:
        x_2 = x_1
        x_1 = x0
        x0 = x1

        matrix = F_der_0(x0) + DividedDifferenceOfOperator(G, x0, x_1) + DividedDifferenceOfOperator(G, x_2, x0) - DividedDifferenceOfOperator(G, x_2, x_1)
        x1 = x0 - np.dot(np.linalg.inv(matrix), H_0(x0))

        if (max(np.abs(x1 - x0)) < eps):
            break
        
        print("{0} Ітерація: Значення H = {1} у точці x = {2}".format(I + 1, H_0(x1), x1))
        I += 1
    
    print('\n\nКількість ітерацій: {0}'.format(I))

    return x0

In [11]:
def PotraOperatorDecompositionMethod(x0, x, F, G, H):
    # element at -1
    x_1 = x0 - 10 ** -4

    #element at -2
    x_2 = x0 - 2 * (10 ** -4)

    I = 0
    matrix = DividedDifferenceOfOperator(H, x0, x_1) + DividedDifferenceOfOperator(H, x_2, x0) - DividedDifferenceOfOperator(H, x_2, x_1)
    
    x1 = x0 - np.dot(np.linalg.inv(matrix), H_0(x0))
    print("{0} Ітерація: Значення H = {1} у точці x = {2}".format(I + 1, H_0(x1), x1))
    I += 1

    while True:
        x_2 = x_1
        x_1 = x0
        x0 = x1

        matrix = DividedDifferenceOfOperator(H, x0, x_1) + DividedDifferenceOfOperator(H, x_2, x0) - DividedDifferenceOfOperator(H, x_2, x_1)
        x1 = x0 - np.dot(np.linalg.inv(matrix), H_0(x0))

        if (max(np.abs(x1 - x0)) < eps):
            break
        
        print("{0} Ітерація: Значення H = {1} у точці x = {2}".format(I + 1, H_0(x1), x1))
        I += 1
    
    print('\n\nКількість ітерацій: {0}'.format(I))

    return x0

In [12]:
print("Метод Ньютона-Потра з декомпозицією оператора\n\n")
a = NewtonPotraOperatorDecompositionMethod(x0, x, F, G, H)
print(a)

Метод Ньютона-Потра з декомпозицією оператора


[0.13333333 0.33333333]
1 Ітерація: Значення H = [0.19111111 0.48762469] у точці x = [0.86666667 0.66666667]
2 Ітерація: Значення H = [-0.07732646  0.08899759] у точці x = [0.88050792 0.4218266 ]
3 Ітерація: Значення H = [-0.0773951   0.06003426] у точці x = [0.77612168 0.56051504]
4 Ітерація: Значення H = [-0.12444276  0.08834402] у точці x = [0.90268686 0.37628161]
5 Ітерація: Значення H = [-0.06581862  0.04780935] у точці x = [0.81413788 0.50415371]
6 Ітерація: Значення H = [-15.81904923  12.75762935] у точці x = [ 2.00006766 -1.19848205]
7 Ітерація: Значення H = [-4.37270178  3.58702349] у точці x = [ 1.43839602 -0.56294768]
8 Ітерація: Значення H = [-1.48113275  0.65311225] у точці x = [ 1.1073381  -0.15356136]
9 Ітерація: Значення H = [-0.27704106  0.44438757] у точці x = [1.05006069 0.21775763]
10 Ітерація: Значення H = [-0.02693133  0.19703301] у точці x = [0.91486389 0.42585131]
11 Ітерація: Значення H = [-0.0746587   0.04265606]

In [13]:
print("Метод Потра з декомпозицією оператора\n\n")
a = PotraOperatorDecompositionMethod(x0, x, F, G, H)
print(a)

Метод Потра з декомпозицією оператора


1 Ітерація: Значення H = [0.1911111  0.48762467] у точці x = [0.86666667 0.66666666]
2 Ітерація: Значення H = [-0.15436227  0.07449539] у точці x = [0.90709555 0.35639578]
3 Ітерація: Значення H = [-0.05909064  0.06350359] у точці x = [0.83858598 0.47761234]
4 Ітерація: Значення H = [-0.5257645   0.35020265] у точці x = [0.55061721 0.88133913]
5 Ітерація: Значення H = [-0.1665715   0.12058008] у точці x = [0.69849452 0.67126523]
6 Ітерація: Значення H = [-0.07068369  0.04142168] у точці x = [0.80630931 0.51109708]
7 Ітерація: Значення H = [-0.39630214  0.36578464] у точці x = [1.04142246 0.18311794]
8 Ітерація: Значення H = [-0.13009692  0.09263149] у точці x = [0.90673549 0.37053924]
9 Ітерація: Значення H = [-0.05918807  0.05907803] у точці x = [0.81347352 0.51199802]
10 Ітерація: Значення H = [-63.94611703  34.77466263] у точці x = [ 2.90260871 -2.34763146]
11 Ітерація: Значення H = [-10.8562263   10.51613682] у точці x = [ 1.8662922  -0.94135